Задание 1. Написать теггер на данных с русским языком
проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации
написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
сравнить все реализованные методы, сделать выводы  


In [1]:
!wget -O ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget -O ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2023-07-11 15:43:53--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40736581 (39M) [text/plain]
Saving to: ‘ru_syntagrus-ud-train.conllu’

ru_syntagrus-ud-tra 100%[===================>]  38.85M   202MB/s    in 0.2s    

2023-07-11 15:43:54 (202 MB/s) - ‘ru_syntagrus-ud-train.conllu’ saved [40736581/40736581]

--2023-07-11 15:43:54--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108

In [2]:
!pip install pyconll

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [4]:
import pandas as pd
import pyconll

In [5]:
import nltk

In [6]:
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger

In [7]:
full_train = pyconll.load_from_file('./ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('./ru_syntagrus-ud-dev.conllu')

In [8]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [9]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [10]:
unigram_tagger = UnigramTagger(fdata_train)

In [11]:
bigram_tagger = BigramTagger(fdata_train)

In [12]:
display(bigram_tagger.evaluate(fdata_test))

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  """Entry point for launching an IPython kernel.


0.6093886320724006

In [21]:
trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
display(trigram_tagger.tag(fdata_sent_test[100]), trigram_tagger.evaluate(fdata_train))

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  


[('Это', 'PRON'),
 ('сочинение', None),
 ('известно', None),
 ('во', 'ADP'),
 ('многих', 'NUM'),
 ('вариантах', None),
 ('(', 'PUNCT'),
 ('самые', 'ADJ'),
 ('ранние', 'ADJ'),
 ('из', 'ADP'),
 ('них', 'PRON'),
 ('почти', 'ADV'),
 ('на', 'ADP'),
 ('сто', 'NUM'),
 ('лет', 'NOUN'),
 ('старше', 'ADJ'),
 (')', 'PUNCT'),
 ('и', 'CCONJ'),
 ('восходит', None),
 ('к', 'ADP'),
 ('ещё', None),
 ('более', 'ADV'),
 ('древним', None),
 ('рукописям', None),
 ('XVI', None),
 ('в', 'ADP'),
 ('.', 'PUNCT')]

0.9722114026401866

In [15]:
!pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.6/356.6 KB 33.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.6.0-py2.py3-none-any.whl size=351305 sha256=4190fcf9f0accb1ecea9bdfaefb1fb6f815f1721c8c11289e989158a8697fdfb
  Stored in directory: /root/.cache/pip/wheels/59/7c/84/b5fafc338acbf8e65d3342168a02e259a366193552d0eced63
Successfully built emoji
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [18]:
from nltk.tag import SequentialBackoffTagger
import emoji

class NamesTagger(SequentialBackoffTagger):
    def __init__(self, *args, **kwargs):
        SequentialBackoffTagger.__init__(self, *args, **kwargs)
        
            
    def choose_tag(self, tokens, index, history):
        word = tokens[index]
       
        if emoji.is_emoji(word.lower()):
             return 'emoji'
        else:
             return None
            
nt = NamesTagger()
print(nt.tag(['\U0001f600'])) 
print(nt.tag(['Adam'])) 
print(nt.tag(['Window']))            

[('😀', 'emoji')]
[('Adam', None)]
[('Window', None)]


In [19]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
#import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [66]:
hvectorizer = HashingVectorizer(n_features=20, analyzer='word')

In [44]:
cvectorizer = CountVectorizer(ngram_range=(1, 3), analyzer='char')

In [28]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(str(tok[0]))
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [ ]:
#tags = [str(t) for t in tag_data["Tags"]]

In [67]:
X_train = hvectorizer.fit_transform(train_tok)
X_test = hvectorizer.transform(test_tok)

In [68]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)

In [69]:
test_enc_labels = le.transform(test_label)

In [70]:
lr = LogisticRegression(random_state=0, max_iter=10)
lr.fit(X_train, train_enc_labels)

/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(max_iter=10, random_state=0)

In [71]:
pred = lr.predict(X_test)
accuracy_score(test_enc_labels, pred)

0.23887622892115373

In [65]:
X_train = cvectorizer.fit_transform(train_tok)
X_test = cvectorizer.transform(test_tok)

lr = LogisticRegression(random_state=0, max_iter=10)
lr.fit(X_train, train_enc_labels)
pred = lr.predict(X_test)
accuracy_score(test_enc_labels, pred)

/shared-libs/python3.7/py/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.7473272999544242

Задание 2. Проверить, насколько хорошо работает NER
Данные брать из Index of /pub/named_entities
проверить NER из nltk/spacy/deeppavlov.
написать свой NER, попробовать разные подходы.
передаём в сетку токен и его соседей.
передаём в сетку только токен.
свой вариант.
сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.


In [74]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2023-07-11 16:40:01--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M  1.79MB/s    in 1.0s    

2023-07-11 16:40:03 (1.79 MB/s) - ‘collection5.zip’ saved [1899530/1899530]



In [75]:
!unzip collection5.zip

  inflating: Collection5/246.txt     
  inflating: Collection5/247.ann     
  inflating: Collection5/247.txt     
  inflating: Collection5/248.ann     
  inflating: Collection5/248.txt     
  inflating: Collection5/249.ann     
  inflating: Collection5/249.txt     
  inflating: Collection5/250.ann     
  inflating: Collection5/250.txt     
  inflating: Collection5/251.ann     
  inflating: Collection5/251.txt     
  inflating: Collection5/252.ann     
  inflating: Collection5/252.txt     
  inflating: Collection5/253.ann     
  inflating: Collection5/253.txt     
  inflating: Collection5/254.ann     
  inflating: Collection5/254.txt     
  inflating: Collection5/255.ann     
  inflating: Collection5/255.txt     
  inflating: Collection5/256.ann     
  inflating: Collection5/256.txt     
  inflating: Collection5/257.ann     
  inflating: Collection5/257.txt     
  inflating: Collection5/258.ann     
  inflating: Collection5/258.txt     
  inflating: Collection5/259.ann     
  inflating:

In [79]:
#!python -m venv env 
# #.\env\Scripts\activate.bat
#!pip install deeppavlov
!python -m deeppavlov install squad_bert

#!python -m deeppavlov install ner_ontonotes

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [80]:

import deeppavlov
from deeppavlov import configs, build_model
deeppavlov_ner = build_model(configs.ner, download=True)
rus_document = "Нью-Йорк, США, 30 апреля 2020, 01:01 — REGNUM В администрации президента США Дональда Трампа планируют пройти все этапы создания вакцины от коронавируса в ускоренном темпе и выпустить 100 млн доз до конца 2020 года, передаёт агентство Bloomberg со ссылкой на осведомлённые источники"
deeppavlov_ner([rus_document])

AttributeError: 'Struct' object has no attribute 'get'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=af133a3e-e74e-4642-80b0-22c9c55ad802' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>